## 1. Limpieza básica del CSV
Primero, vamos a cargar y limpiar el archivo CSV. Como no veo el contenido exacto, te muestro un ejemplo general en Python usando pandas:

In [11]:
#!pip install pandas
import pandas as pd

In [12]:
df = pd.read_csv('./data/case-study-data.csv')
df.head()

,DATE,ANONYMIZED CATEGORY,ANONYMIZED PRODUCT,ANONYMIZED BUSINESS,ANONYMIZED LOCATION,QUANTITY,UNIT PRICE
0,"August 18, 2024, 9:32 PM",Category-106,Product-21f4,Business-de42,Location-1ba8,1,850
1,"August 18, 2024, 9:32 PM",Category-120,Product-4156,Business-de42,Location-1ba8,2,"1,910"
2,"August 18, 2024, 9:32 PM",Category-121,Product-49bd,Business-de42,Location-1ba8,1,"3,670"
3,"August 18, 2024, 9:32 PM",Category-76,Product-61dd,Business-de42,Location-1ba8,1,"2,605"
4,"August 18, 2024, 9:32 PM",Category-119,Product-66e0,Business-de42,Location-1ba8,5,"1,480"


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333405 entries, 0 to 333404
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   DATE                 333405 non-null  object
 1   ANONYMIZED CATEGORY  333405 non-null  object
 2   ANONYMIZED PRODUCT   333405 non-null  object
 3   ANONYMIZED BUSINESS  333405 non-null  object
 4   ANONYMIZED LOCATION  333405 non-null  object
 5   QUANTITY             333405 non-null  int64 
 6   UNIT PRICE           333397 non-null  object
dtypes: int64(1), object(6)
memory usage: 17.8+ MB


In [14]:
df.isna().sum()

DATE                   0
ANONYMIZED CATEGORY    0
ANONYMIZED PRODUCT     0
ANONYMIZED BUSINESS    0
ANONYMIZED LOCATION    0
QUANTITY               0
UNIT PRICE             8
dtype: int64

In [15]:
# Limpieza básica:
# - Eliminar filas duplicadas
df.drop_duplicates(inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 329881 entries, 0 to 333404
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   DATE                 329881 non-null  object
 1   ANONYMIZED CATEGORY  329881 non-null  object
 2   ANONYMIZED PRODUCT   329881 non-null  object
 3   ANONYMIZED BUSINESS  329881 non-null  object
 4   ANONYMIZED LOCATION  329881 non-null  object
 5   QUANTITY             329881 non-null  int64 
 6   UNIT PRICE           329873 non-null  object
dtypes: int64(1), object(6)
memory usage: 20.1+ MB


In [17]:
# - Eliminar filas con valores nulos en columnas clave (ajusta 'columna_clave' según tu dataset)
df = df.dropna(subset=['UNIT PRICE'])

In [18]:
df.isna().sum()

DATE                   0
ANONYMIZED CATEGORY    0
ANONYMIZED PRODUCT     0
ANONYMIZED BUSINESS    0
ANONYMIZED LOCATION    0
QUANTITY               0
UNIT PRICE             0
dtype: int64

In [19]:
# - Opcional: renombrar columnas para quitar espacios o caracteres especiales
df.columns = [col.strip().replace(' ', '_').lower() for col in df.columns]
df.head()

,date,anonymized_category,anonymized_product,anonymized_business,anonymized_location,quantity,unit_price
0,"August 18, 2024, 9:32 PM",Category-106,Product-21f4,Business-de42,Location-1ba8,1,850
1,"August 18, 2024, 9:32 PM",Category-120,Product-4156,Business-de42,Location-1ba8,2,"1,910"
2,"August 18, 2024, 9:32 PM",Category-121,Product-49bd,Business-de42,Location-1ba8,1,"3,670"
3,"August 18, 2024, 9:32 PM",Category-76,Product-61dd,Business-de42,Location-1ba8,1,"2,605"
4,"August 18, 2024, 9:32 PM",Category-119,Product-66e0,Business-de42,Location-1ba8,5,"1,480"


In [16]:
# Guardar el archivo limpio
df.to_csv('./data/case-study-data-clean.csv', index=False)

## 2. Conectar y cargar a Snowflake
Necesitas instalar el conector de Snowflake para Python:



In [ ]:
#pip install snowflake-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 36.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [snowflake-connector-python]-connector-python]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import snowflake.connector

In [30]:
# Conexión a Snowflake (solo necesitas usuario, contraseña y cuenta para crear recursos)
conn = snowflake.connector.connect(
    user='VIWITA4266',
    password='SupermanRafty26lego10',
    account='USPZCGH-US48484'
)
cur = conn.cursor()

In [4]:
# Crear warehouse (si no existe)
cur.execute("""
CREATE WAREHOUSE IF NOT EXISTS my_wh
  WAREHOUSE_SIZE = 'XSMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE;
""")

In [5]:
# Crear base de datos (si no existe)
cur.execute("CREATE DATABASE IF NOT EXISTS my_db;")

In [6]:
# Crear esquema (si no existe)
cur.execute("CREATE SCHEMA IF NOT EXISTS my_db.my_schema;")

In [31]:
# Usar los recursos recién creados
cur.execute("USE WAREHOUSE my_wh;")
cur.execute("USE DATABASE my_db;")
cur.execute("USE SCHEMA my_schema;")

In [ ]:
import snowflake.connector
import pandas as pd

# Cargar el CSV limpio
df = pd.read_csv('./data/case-study-data-clean.csv')

# Conexión a Snowflake
conn = snowflake.connector.connect(
    user='TU_USUARIO',
    password='TU_CONTRASEÑA',
    account='TU_CUENTA',  # ejemplo: abcd1234.eu-west-1
    warehouse='TU_WAREHOUSE',
    database='TU_DATABASE',
    schema='TU_SCHEMA'
)

# Crear un cursor
cur = conn.cursor()

# Crear tabla (ajusta los tipos de datos según tu CSV)
create_table_query = """
CREATE OR REPLACE TABLE case_study_data (
    columna1 VARCHAR,
    columna2 INTEGER,
    columna3 DATE
    -- agrega más columnas según tu CSV
);
"""
cur.execute(create_table_query)

# Subir datos usando pandas
from snowflake.connector.pandas_tools import write_pandas

success, nchunks, nrows, _ = write_pandas(conn, df, 'case_study_data')
print(f"Subidos {nrows} registros a Snowflake.")

cur.close()
conn.close()

1. Define la estructura de la tabla según tu CSV limpio
Por tu dataset, la tabla podría ser así:

In [8]:
create_table_query = """
CREATE OR REPLACE TABLE case_study_data (
    date TIMESTAMP,
    anonymized_category VARCHAR,
    anonymized_product VARCHAR,
    anonymized_business VARCHAR,
    anonymized_location VARCHAR,
    quantity INTEGER,
    unit_price FLOAT
);
"""
cur.execute(create_table_query)

2. Carga los datos con write_pandas
Asegúrate de que tu DataFrame (df) tenga los mismos nombres de columnas (en minúsculas y con guiones bajos, como en la tabla).

In [9]:
# En una celda de Jupyter:
#!pip install pyarrow

In [21]:
cur.execute("USE WAREHOUSE my_wh;")
cur.execute("USE DATABASE my_db;")
cur.execute("USE SCHEMA my_schema;")

In [23]:
df.head()

,date,anonymized_category,anonymized_product,anonymized_business,anonymized_location,quantity,unit_price
0,"August 18, 2024, 9:32 PM",Category-106,Product-21f4,Business-de42,Location-1ba8,1,850
1,"August 18, 2024, 9:32 PM",Category-120,Product-4156,Business-de42,Location-1ba8,2,"1,910"
2,"August 18, 2024, 9:32 PM",Category-121,Product-49bd,Business-de42,Location-1ba8,1,"3,670"
3,"August 18, 2024, 9:32 PM",Category-76,Product-61dd,Business-de42,Location-1ba8,1,"2,605"
4,"August 18, 2024, 9:32 PM",Category-119,Product-66e0,Business-de42,Location-1ba8,5,"1,480"


In [24]:
cur.execute("SELECT CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA();")
print(cur.fetchone())

('MY_WH', 'MY_DB', 'MY_SCHEMA')


In [25]:
cur.execute("SHOW TABLES;")
for row in cur.fetchall():
    print(row)

(datetime.datetime(2025, 7, 18, 10, 19, 29, 133000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'CASE_STUDY_DATA', 'MY_DB', 'MY_SCHEMA', 'TABLE', '', '', 0, 0, 'ACCOUNTADMIN', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', 'N', 'N', 'N', 'N')


In [26]:
print(df.columns)

Index(['date', 'anonymized_category', 'anonymized_product',
       'anonymized_business', 'anonymized_location', 'quantity', 'unit_price'],
      dtype='object')


In [27]:
print(df.dtypes)

date                   object
anonymized_category    object
anonymized_product     object
anonymized_business    object
anonymized_location    object
quantity                int64
unit_price             object
dtype: object


In [28]:
import pandas as pd

# Convertir 'date' a datetime
df['date'] = pd.to_datetime(df['date'])

# Convertir 'unit_price' a float (elimina comas si las hay)
df['unit_price'] = df['unit_price'].astype(str).str.replace(',', '').astype(float)

# (Opcional) Asegúrate de que 'quantity' sea int
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').astype('Int64')

# Verifica los tipos
print(df.dtypes)

/var/folders/nx/9lp38gjd74s6_19k7c_d2mq40000gn/T/ipykernel_63663/184771511.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


date                   datetime64[ns]
anonymized_category            object
anonymized_product             object
anonymized_business            object
anonymized_location            object
quantity                        Int64
unit_price                    float64
dtype: object


In [35]:
# Cambia el nombre de la columna en el DataFrame
df = df.rename(columns={'date': 'fecha'})

# Crea la tabla con el nuevo nombre
create_table_query = """
CREATE OR REPLACE TABLE CASE_STUDY_DATA (
    fecha TIMESTAMP,
    anonymized_category VARCHAR,
    anonymized_product VARCHAR,
    anonymized_business VARCHAR,
    anonymized_location VARCHAR,
    quantity INTEGER,
    unit_price FLOAT
);
"""
cur.execute(create_table_query)

In [37]:
df.columns = [col.upper() for col in df.columns]
print(df.columns)

Index(['FECHA', 'ANONYMIZED_CATEGORY', 'ANONYMIZED_PRODUCT',
       'ANONYMIZED_BUSINESS', 'ANONYMIZED_LOCATION', 'QUANTITY', 'UNIT_PRICE'],
      dtype='object')


In [38]:
create_table_query = """
CREATE OR REPLACE TABLE CASE_STUDY_DATA (
    FECHA TIMESTAMP,
    ANONYMIZED_CATEGORY VARCHAR,
    ANONYMIZED_PRODUCT VARCHAR,
    ANONYMIZED_BUSINESS VARCHAR,
    ANONYMIZED_LOCATION VARCHAR,
    QUANTITY INTEGER,
    UNIT_PRICE FLOAT
);
"""
cur.execute(create_table_query)

In [39]:
from snowflake.connector.pandas_tools import write_pandas

success, nchunks, nrows, _ = write_pandas(conn, df, 'CASE_STUDY_DATA')
print(f"Subidos {nrows} registros a Snowflake.")

/var/folders/nx/9lp38gjd74s6_19k7c_d2mq40000gn/T/ipykernel_63663/1740773204.py:3: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, df, 'CASE_STUDY_DATA')


Subidos 329873 registros a Snowflake.


3. Verifica los datos en Snowflake
Puedes ejecutar en la consola web o desde Python:

In [41]:
#SELECT * FROM case_study_data LIMIT 10;